# Намаляване на историята на чата с помощта на Scratchpad на агент в Semantic Kernel

Тази тетрадка демонстрира как да използвате функцията за намаляване на историята на чата в Semantic Kernel заедно със Scratchpad на агент, за да поддържате контекст в разговорите. Това е от съществено значение за изграждането на ефективни AI агенти, които могат да се справят с дълги разговори, без да надвишават лимитите за токени.

## Какво ще научите:
1. **Намаляване на историята на чата**: Как автоматично да обобщавате историята на разговорите, за да управлявате използването на токени
2. **Scratchpad на агент**: Система за постоянна памет за проследяване на предпочитанията на потребителя и завършените задачи
3. **Проследяване на използването на токени**: Наблюдение на промените в използването на токени със и без намаляване на историята

## Предпоставки:
- Настройка на Azure OpenAI с конфигурирани променливи на средата
- Разбиране на основни концепции за агенти от предишни уроци


## Импортиране на необходимите пакети


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## Разбиране на Agent Scratchpad

### Какво е Agent Scratchpad?

**Agent Scratchpad** е система за постоянна памет, която агентите използват, за да:
- **Следят изпълнените задачи**: Записват какво е направено за потребителя
- **Съхраняват предпочитанията на потребителя**: Запомнят харесвания, нехаресвания и изисквания
- **Поддържат контекст**: Съхраняват важна информация, която е достъпна през различни разговори
- **Намаляват излишъка**: Избягват задаването на едни и същи въпроси многократно

### Как работи:
1. **Операции за запис**: Агентът актуализира scratchpad-а след като научи нова информация
2. **Операции за четене**: Агентът се консултира със scratchpad-а при вземане на решения
3. **Постоянство**: Информацията остава налична, дори когато историята на чата е намалена

Мислете за това като за личен бележник на агента, който допълва историята на разговора.


## Конфигурация на средата


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## Създаване на плъгин за Scratchpad на агента

Този плъгин позволява на агента да чете и записва в постоянен файл за scratchpad.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## Инициализиране на Редуктор за История на Чата

ChatHistorySummarizationReducer автоматично обобщава историята на разговора, когато тя надвиши определен праг.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## Създаване на агент за планиране на почивки

Този агент ще помага на потребителите да планират почивки, като запазва контекста чрез scratchpad.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## Провеждане на разговор за планиране на ваканция

Сега нека преминем през цялостен разговор, демонстриращ:
1. Първоначална заявка за планиране
2. Събиране на предпочитания
3. Създаване на маршрут
4. Промяна на местоположението
5. Намаляване на историята на чата
6. Използване на бележник


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## Анализ на резултатите

Нека анализираме какво се случи по време на нашия разговор:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## Основни моменти

### 1. Намаляване на историята на чата
- **Автоматично задействане**: Намаляването се случва, когато броят на съобщенията надвиши прага
- **Спестяване на токени**: Значително намаляване на използването на токени след обобщение
- **Запазване на контекста**: Важната информация се съхранява в обобщенията

### 2. Ползи от бележника на агента
- **Постоянна памет**: Предпочитанията на потребителя се запазват след намаляване на историята
- **Проследяване на задачи**: Агента поддържа запис на завършената работа
- **Подобрено изживяване**: Няма нужда от повторение на предпочитанията

### 3. Модели на използване на токени
- **Линеен растеж**: Токените се увеличават с всяко съобщение
- **Драматичен спад**: Намаляването значително намалява броя на токените
- **Устойчиви разговори**: Позволява по-дълги взаимодействия в рамките на ограниченията


## Почистване

Изчистете файла за чернова, създаден по време на тази демонстрация:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# Резюме

Поздравления! Успешно създадохте AI агент с напреднали възможности за управление на контекста:

## Какво научихте:
- **Намаляване на историята на чата**: Автоматично обобщаване на разговори за управление на лимитите на токените
- **Работен бележник на агента**: Реализиране на постоянна памет за предпочитанията на потребителя и изпълнените задачи
- **Управление на токени**: Проследяване и оптимизиране на използването на токени в дълги разговори
- **Запазване на контекста**: Поддържане на важна информация при намаляване на разговорите

## Приложения в реалния свят:
- **Ботове за обслужване на клиенти**: Запомняне на предпочитанията на клиентите между сесиите
- **Лични асистенти**: Проследяване на текущи проекти и навици на потребителя
- **Образователни наставници**: Поддържане на напредъка на учениците и техните предпочитания за учене
- **Асистенти в здравеопазването**: Съхраняване на историята на пациентите, като същевременно се спазват лимитите на токените

## Следващи стъпки:
- Реализирайте по-сложни схеми за работния бележник
- Добавете съхранение в база данни за сценарии с множество потребители
- Създайте персонализирани стратегии за намаляване за специфични домейни
- Комбинирайте с векторни бази данни за семантично търсене в паметта
- Създайте агенти, които могат да възобновяват разговори дни по-късно с пълен контекст



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да било недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
